---
---
#### Daniel Bruintjies
#### Sept 2023
___
### ZINDI COMPETITION
___
# Network Traffic Scenario Prediction Challenge by ITU
---
#### The objective of this challenge is to build a model based on the training set data to predict the traffic scenario for unknown traffic at each moment

https://zindi.africa/competitions/network-traffic-scenario-prediction-challenge

## Imports

In [1]:
import os
import gc
gc.enable()
import glob
import random
import numpy as np 
import pandas as pd  
import json
from tqdm import tqdm
from sklearn.preprocessing import RobustScaler,StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import lr_scheduler
from tensorflow.keras.utils import pad_sequences
from sklearn.metrics import accuracy_score
import lzma
import pickle
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 150)

def set_seeed(seed_value=23, use_cuda=True):
    np.random.seed(seed_value) 
    torch.manual_seed(seed_value) 
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) 
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seeed()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


cuda:0


## Paths & Settings

In [2]:
class CFG:
    # Precompute data for model input, leave as True
    CREATE_DATA = True
    
    sequence_length = 5000
    
    # If data was precomputed previously pass root to data here
    saved_path = ''

# Train Files Path
train_files_p = '/kaggle/input/network-traffic-scenario-prediction/Train_data/Train_data'
# Test Files Path
test_files_p = '/kaggle/input/network-traffic-scenario-prediction/Test_data/Test_data'
# Sample Submission File Path
ss_p = '/kaggle/input/network-traffic-scenario-prediction/SampleSubmission.csv'

## Load Dataframes

In [3]:
%%time

def reader(f):
    try:
        df = pd.read_csv(f)
        df['ID']=f.split('/')[-1].split('.')[0]
        return df
    except: pass
    
train_files = glob.glob(train_files_p+'/**')
dfs = []
for f in tqdm(train_files):
    dfs.append(reader(f))
train = pd.concat(dfs).fillna(0)
del dfs
gc.collect()


test_files = glob.glob(test_files_p+'/**')
dfs = []
for f in tqdm(test_files):
    dfs.append(reader(f))
test = pd.concat(dfs).fillna(0)
del dfs
gc.collect()

ss = pd.read_csv(ss_p)

display(train, test, ss)

100%|██████████| 19/19 [00:01<00:00, 12.73it/s]


,time,portPktIn,portPktOut,qSize,label,ID
0,0,0.0,0.0,0,8,Train74
1,1,0.0,0.0,0,8,Train74
2,2,0.0,0.0,0,8,Train74
3,3,186770.0,186770.0,0,8,Train74
4,4,0.0,0.0,0,8,Train74
...,...,...,...,...,...,...
104995,104995,1201796.0,1249580.0,2652327,7,Train44
104996,104996,1183244.0,1249897.0,2585674,7,Train44
104997,104997,1245195.0,1249337.0,2581532,7,Train44
104998,104998,1205199.0,1250295.0,2536436,7,Train44


,time,portPktIn,portPktOut,qSize,ID
0,0,0.0,0.0,0,Test10
1,1,111934.0,111934.0,0,Test10
2,2,34623.0,34623.0,0,Test10
3,3,0.0,0.0,0,Test10
4,4,1012.0,1012.0,0,Test10
...,...,...,...,...,...
128995,128995,967479.0,956083.0,11396,Test0
128996,128996,1249434.0,1249434.0,11396,Test0
128997,128997,1182864.0,1153987.0,40273,Test0
128998,128998,1155450.0,1185363.0,10360,Test0


,ID,Target
0,test0_0,0
1,test0_1,0
2,test0_2,0
3,test0_3,0
4,test0_4,0
...,...,...
2337995,test18_144995,0
2337996,test18_144996,0
2337997,test18_144997,0
2337998,test18_144998,0


CPU times: user 6.94 s, sys: 1.83 s, total: 8.77 s
Wall time: 12.4 s


In [4]:
display(train['ID'].value_counts())

ID
Train59    149000
Train74    148000
Train61    148000
Train18    148000
Train13    147000
            ...  
Train23    103000
Train71    102000
Train20    102000
Train22    100000
Train65    100000
Name: count, Length: 78, dtype: int64

In [5]:
display(test['ID'].value_counts())

ID
Test18    145000
Test12    144000
Test6     141000
Test14    139000
Test16    137000
Test2     136000
Test3     131000
Test0     129000
Test15    128000
Test10    122000
Test9     119000
Test7     117000
Test1     113000
Test4     108000
Test11    108000
Test5     107000
Test13    106000
Test8     105000
Test17    103000
Name: count, dtype: int64

In [6]:
display(train['label'].nunique())

12

## Create Train / Val Sequence Arrays

In [7]:
# Create Train / Val Splits by Scenario ('ID')
train_ids = list(train['ID'].unique())
train_ids, val_ids = train_test_split(train_ids, random_state=1, test_size=0.15)
print(len(train_ids), len(val_ids))

# Find Sequence Splits
sequence_length = CFG.sequence_length
train = train.sort_values('time').reset_index(drop=True)

# Create Time Buckets
train['sequence_id'] = train['time'] // sequence_length
# Create Time Bucket Identifier: 
train['sequence_id'] = train['sequence_id'].astype(str)
train['sequence_id_ID'] = train['sequence_id'] + '_' + train['ID']
display(train)

labels = np.array(list(set(train['label'].unique())))
print(labels)

66 12


,time,portPktIn,portPktOut,qSize,label,ID,sequence_id,sequence_id_ID
0,0,0.0,0.0,0,8,Train74,0,0_Train74
1,0,542864.0,542864.0,0,3,Train9,0,0_Train9
2,0,0.0,0.0,0,9,Train32,0,0_Train32
3,0,0.0,0.0,0,0,Train53,0,0_Train53
4,0,0.0,0.0,0,11,Train57,0,0_Train57
...,...,...,...,...,...,...,...,...
9657995,148995,0.0,0.0,0,10,Train59,29,29_Train59
9657996,148996,0.0,0.0,0,10,Train59,29,29_Train59
9657997,148997,0.0,0.0,0,10,Train59,29,29_Train59
9657998,148998,0.0,0.0,0,10,Train59,29,29_Train59


[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [8]:
%%time

# Create Train / Val dataframes
train_df = train.copy()
xtrain_df = train_df[train_df['ID'].isin(train_ids)].reset_index(drop=True)
xval_df = train_df[train_df['ID'].isin(val_ids)].reset_index(drop=True)

# Scale Features
features = ['time','portPktIn','portPktOut','qSize']
scaler = RobustScaler() 
xtrain_df[features] = scaler.fit_transform(xtrain_df[features])
xval_df[features] = scaler.transform(xval_df[features])

xtrain_df.fillna(0,inplace=True)
xval_df.fillna(0,inplace=True)

xtrain_df.replace([np.inf,-np.inf],0,inplace=True)
xval_df.replace([np.inf,-np.inf],0,inplace=True)

del train,train_df
gc.collect()

if CFG.CREATE_DATA:
    
    # Extract and Pad Train Sequences At All Time Bucket Identifiers
    xtrain_idids = xtrain_df['sequence_id_ID'].unique()
    xtrain_data = []
    xtrain_targets = []
    for d in tqdm(xtrain_idids):
        data = xtrain_df[xtrain_df['sequence_id_ID']==d].sort_values('time')
        inputs = data[features].values
        inputs = pad_sequences([inputs], maxlen=sequence_length, dtype='float', 
                               padding='post', value=-1)[0,:,:]
        xtrain_data.append(inputs)

        targets = data['label'].values
        targets = pad_sequences([targets], maxlen=sequence_length, dtype='int', 
                                padding='post', value=max(labels)+1)[0]
        xtrain_targets.append(targets)
    
    
    # Extract and Pad Val Sequences At All Time Bucket Identifiers
    xval_idids = xval_df['sequence_id_ID'].unique()
    xval_data = []
    xval_targets = []
    for d in tqdm(xval_idids):
        data = xval_df[xval_df['sequence_id_ID']==d].sort_values('time')
        inputs = data[features].values
        inputs = pad_sequences([inputs], maxlen=sequence_length, dtype='float', 
                               padding='post', value=-1)[0,:,:]
        xval_data.append(inputs)

        targets = data['label'].values
        targets = pad_sequences([targets], maxlen=sequence_length, dtype='int', 
                                padding='post', value=max(labels)+1)[0]
        xval_targets.append(targets)

        
    xtrain_data = np.array(xtrain_data)
    xval_data = np.array(xval_data)

    xtrain_targets = np.array(xtrain_targets)
    xval_targets = np.array(xval_targets)
    
    
    with open('xtrain_data.npy','wb') as f:
        np.save(f, xtrain_data)

    with open('xval_data.npy','wb') as f:
        np.save(f, xval_data)

    with open('xtrain_targets.npy','wb') as f:
        np.save(f, xtrain_targets)

    with open('xval_targets.npy','wb') as f:
        np.save(f, xval_targets)

else:
    
    saved_path = CFG.saved_path
    
    with open(saved_path+'xtrain_data.npy','rb') as f:
        xtrain_data = np.load(f)

    with open(saved_path+'xval_data.npy','rb') as f:
        xval_data = np.load(f)

    with open(saved_path+'xtrain_targets.npy','rb') as f:
        xtrain_targets = np.load(f)

    with open(saved_path+'xval_targets.npy','rb') as f:
        xval_targets = np.load(f)

100%|██████████| 292/292 [00:45<00:00,  6.38it/s]


CPU times: user 16min 22s, sys: 6.06 s, total: 16min 28s
Wall time: 16min 27s


## Modelling

In [9]:
class MySimpleSequenceDataset(Dataset):
    """ Training Dataloader"""
    def __init__(self, data, targets):
        
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.data[idx]
        targets = self.targets[idx]
        
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.long)
    
class MySimpleSequenceDatasetInference(Dataset):
    """ Inference Dataloader"""
    def __init__(self, data):
        
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs = self.data[idx]
        
        return torch.tensor(inputs, dtype=torch.float32)
    
def save_checkpoint(checkpoint, filename):
    torch.save(checkpoint, filename)
    print(f"\n--> Saved checkpoint: {filename.split('.')[0]}")

def load_checkpoint(filename, model):
    model.load_state_dict(torch.load(filename)['state_dict'])
    return model

class MyRNNModel(nn.Module):
    def __init__(self, input_size, num_classes, hidden_size, num_layers):
        super().__init__()

        self.Lstm_layer_1 = nn.GRU(input_size=input_size,
                                    hidden_size=hidden_size,
                                    num_layers=num_layers,
                                    bidirectional=True,
                                    batch_first=True)

        self.Output = nn.Linear(in_features=self.Lstm_layer_1.hidden_size*2, out_features=num_classes)

    def forward(self, inputs):
        lstm_1_seq, _ = self.Lstm_layer_1(inputs)
        output = self.Output(lstm_1_seq)
        return output

In [10]:
batch_size = 16

train_ds = MySimpleSequenceDataset(xtrain_data, xtrain_targets)
val_ds = MySimpleSequenceDataset(xval_data, xval_targets)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

torch.cuda.empty_cache()
gc.collect()

0

In [11]:
%%time
num_classes = len(labels)+1
input_size = len(features)  
hidden_size = 16
num_layers = 1   

model = MyRNNModel(input_size, num_classes, hidden_size, num_layers).to(device)

criterion = nn.CrossEntropyLoss().to(device)

for x,y in train_loader:
    break
    
output = model(x.to(device))
loss = criterion(output.view(-1, num_classes), y.to(device).view(-1))
print(loss)
_, predicted = torch.max(output, 2)
print(output.shape)
print(predicted)
print(predicted.shape)

tensor(2.6131, device='cuda:0', grad_fn=<NllLossBackward0>)
torch.Size([16, 5000, 13])
tensor([[3, 3, 3,  ..., 3, 3, 3],
        [2, 2, 2,  ..., 2, 2, 2],
        [3, 3, 3,  ..., 3, 3, 3],
        ...,
        [9, 9, 9,  ..., 2, 9, 9],
        [3, 3, 3,  ..., 2, 2, 2],
        [2, 2, 2,  ..., 3, 3, 3]], device='cuda:0')
torch.Size([16, 5000])
CPU times: user 2.01 s, sys: 1.35 s, total: 3.35 s
Wall time: 7.86 s


## Model Training

In [12]:
%%time

epochs = 200
batch_size = 4
hidden_size = 64
num_layers = 3

train_ds = MySimpleSequenceDataset(xtrain_data, xtrain_targets)
val_ds = MySimpleSequenceDataset(xval_data, xval_targets)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

torch.cuda.empty_cache()
gc.collect()

input_size = len(features)  
num_classes = len(labels)+1
model = MyRNNModel(input_size, num_classes, hidden_size, num_layers).to(device)

model_filename = 'network_traffic_model.pth'
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=4, verbose=True, min_lr=1e-6)

best_val_accuracy = 0
best_val_targets = []
best_val_preds = []
epochs_without_improvement = 0
patience = 10

for epoch in range(epochs):
    dataset_size = 0
    running_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train ')
    for step,(inputs, targets) in pbar:
        inputs,targets = inputs.to(device), targets.to(device)

        batch_size = inputs.size(0)
        
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, num_classes), targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix(train_loss=f'{epoch_loss:0.4f}',
                         lr=f'{current_lr:0.5f}',
                         gpu_mem=f'{mem:0.2f} GB')
        
    torch.cuda.empty_cache()
    gc.collect()

    model.eval()
    with torch.no_grad():
        all_predictions = []
        all_targets = []
        
        dataset_size = 0
        running_loss = 0.0
        pbar = tqdm(enumerate(val_loader), total=len(val_loader), desc='Valid ')
        for step, (inputs, targets) in pbar:
            inputs,targets = inputs.to(device), targets.to(device)
            batch_size = inputs.size(0)
            
            outputs = model(inputs)
            
            loss = criterion(outputs.view(-1, num_classes), targets.view(-1))
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size

            epoch_loss = running_loss / dataset_size

            mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
            current_lr = optimizer.param_groups[0]['lr']
            pbar.set_postfix(val_loss=f'{epoch_loss:0.4f}',
                             lr=f'{current_lr:0.5f}',
                             gpu_mem=f'{mem:0.2f} GB')
            
            _, predicted = torch.max(outputs, 2)
            all_predictions.extend(predicted.cpu().numpy().ravel())
            all_targets.extend(targets.cpu().numpy().ravel())
    
    all_targets_ = np.array(all_targets)
    all_predictions_ = np.array(all_predictions)

    all_predictions_ = all_predictions_[np.where(all_targets_!=12)]
    all_targets_ = all_targets_[np.where(all_targets_!=12)]

    val_accuracy = accuracy_score(all_targets_, all_predictions_)
    print(f"Epoch {epoch+1}/{epochs}, Accuracy Score: {val_accuracy:.4f}")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_val_targets = all_targets_
        best_val_preds = all_predictions_
        epochs_without_improvement = 0
        checkpoint = {'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
        save_checkpoint(checkpoint=checkpoint, filename=model_filename)
    else:
        epochs_without_improvement +=1

    if epochs_without_improvement == patience:
        break
            
    scheduler.step(val_accuracy)
    
    torch.cuda.empty_cache()
    gc.collect()
    
del xtrain_data, xtrain_targets, xval_data, xval_targets
torch.cuda.empty_cache()
gc.collect()

Valid : 100%|██████████| 73/73 [00:02<00:00, 25.20it/s, gpu_mem=0.39 GB, lr=0.00100, val_loss=1.0213]


Epoch 1/200, Accuracy Score: 0.5963

--> Saved checkpoint: network_traffic_model


Valid : 100%|██████████| 73/73 [00:02<00:00, 25.37it/s, gpu_mem=0.39 GB, lr=0.00100, val_loss=0.9241]


Epoch 2/200, Accuracy Score: 0.6318

--> Saved checkpoint: network_traffic_model


Valid : 100%|██████████| 73/73 [00:02<00:00, 25.55it/s, gpu_mem=0.39 GB, lr=0.00100, val_loss=0.8104]


...

Valid : 100%|██████████| 73/73 [00:03<00:00, 24.28it/s, gpu_mem=0.39 GB, lr=0.00049, val_loss=0.5555]


Epoch 29/200, Accuracy Score: 0.7550


Valid : 100%|██████████| 73/73 [00:02<00:00, 25.66it/s, gpu_mem=0.39 GB, lr=0.00049, val_loss=0.5617]


Epoch 30/200, Accuracy Score: 0.7650
CPU times: user 22min 36s, sys: 9.03 s, total: 22min 45s
Wall time: 22min 46s


190

In [13]:
print('BEST VAL ACCURACY: ', np.round(best_val_accuracy,5))

BEST VAL ACCURACY:  0.76937


## Inference

In [14]:
test_files = glob.glob(test_files_p+'/**')
dfs = []
for f in tqdm(test_files):
    dfs.append(reader(f))
test = pd.concat(dfs).fillna(0)
del dfs
gc.collect()

test['time'] = test['time'].astype(int).astype(str)
test['ID2'] = test['ID'].str.replace('T','t') +"_"+ test['time'] 
test['time'] = test['time'].astype(int)

display(test)

100%|██████████| 19/19 [00:00<00:00, 22.01it/s]


,time,portPktIn,portPktOut,qSize,ID,ID2
0,0,0.0,0.0,0,Test10,test10_0
1,1,111934.0,111934.0,0,Test10,test10_1
2,2,34623.0,34623.0,0,Test10,test10_2
3,3,0.0,0.0,0,Test10,test10_3
4,4,1012.0,1012.0,0,Test10,test10_4
...,...,...,...,...,...,...
128995,128995,967479.0,956083.0,11396,Test0,test0_128995
128996,128996,1249434.0,1249434.0,11396,Test0,test0_128996
128997,128997,1182864.0,1153987.0,40273,Test0,test0_128997
128998,128998,1155450.0,1185363.0,10360,Test0,test0_128998


In [15]:
test_df = test.copy()
test_df['sequence_id'] = test_df['time'] // sequence_length
test_df['sequence_id'] = test_df['sequence_id'].astype(str)
test_df['sequence_id_ID'] = test_df['sequence_id'] + '_' + test_df['ID']
test_df[features] = scaler.transform(test_df[features])
display(test_df)

,time,portPktIn,portPktOut,qSize,ID,ID2,sequence_id,sequence_id_ID
0,-0.999984,-2.178478,-2.241822,-0.217971,Test10,test10_0,0,0_Test10
1,-0.999968,-1.978822,-2.040946,-0.217971,Test10,test10_1,0,0_Test10
2,-0.999952,-2.116721,-2.179688,-0.217971,Test10,test10_2,0,0_Test10
3,-0.999936,-2.178478,-2.241822,-0.217971,Test10,test10_3,0,0_Test10
4,-0.999920,-2.176673,-2.240006,-0.217971,Test10,test10_4,0,0_Test10
...,...,...,...,...,...,...,...,...
128995,1.070961,-0.452794,-0.526038,-0.018164,Test0,test0_128995,25,25_Test0
128996,1.070977,0.050127,0.000409,-0.018164,Test0,test0_128996,25,25_Test0
128997,1.070993,-0.068613,-0.170880,0.488139,Test0,test0_128997,25,25_Test0
128998,1.071009,-0.117511,-0.114572,-0.036329,Test0,test0_128998,25,25_Test0


In [16]:
xtest_idids = test_df['sequence_id_ID'].unique()

xtest_data = []
test_id_data = []
for d in tqdm(xtest_idids):
    data = test_df[test_df['sequence_id_ID']==d].sort_values('time')
    inputs = data[features].values
    inputs = pad_sequences([inputs], maxlen=sequence_length, dtype='float', 
                           padding='post', value=-1)[0,:,:]
    xtest_data.append(inputs)
    
    test_id_data_ = data['ID2'].values
    test_id_data_ = pad_sequences([test_id_data_], maxlen=sequence_length, dtype=object, 
                                  padding='post', value='0')[0]
    test_id_data.append(test_id_data_)
    
xtest_data = np.array(xtest_data)
test_id_data = np.array(test_id_data)

100%|██████████| 476/476 [01:18<00:00,  6.09it/s]


In [17]:
%%time

model_filename = f'network_traffic_model.pth'
batch_size = 16

test_ds = MySimpleSequenceDatasetInference(xtest_data)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

hidden_size = 64
num_layers = 3
input_size = len(features)  
num_classes = len(labels)+1
model = MyRNNModel(input_size, num_classes, hidden_size, num_layers).to(device)          
model = load_checkpoint(filename=model_filename, model=model)

torch.cuda.empty_cache()
gc.collect()

CPU times: user 246 ms, sys: 2.99 ms, total: 249 ms
Wall time: 247 ms


9

In [18]:
all_predictions = []
model.eval()
with torch.no_grad():
    pbar = tqdm(enumerate(test_loader), total=len(test_loader), desc='Inference ')
    for step, inputs in pbar:
        inputs = inputs.to(device)
        outputs = model(inputs)
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(gpu_mem=f'{mem:0.2f} GB')
            
        _, predicted = torch.max(outputs, 2)
        all_predictions.extend(predicted.cpu().numpy().ravel())

Inference : 100%|██████████| 30/30 [00:01<00:00, 23.35it/s, gpu_mem=0.73 GB]


In [19]:
test_id_data_ = test_id_data.ravel()
all_predictions_ = np.array(all_predictions)

sub = pd.DataFrame(all_predictions_)
sub['ID'] = test_id_data_
sub.columns = ['Target','ID']
sub = sub[['ID','Target']]

sub = sub[sub['ID']!='0']
sub.loc[sub['Target']==12,'Target'] = 8
sub_ = pd.merge(ss[['ID']],sub[['ID', 'Target']], how='left', on='ID')

display(sub_)
display(sub_['Target'].value_counts())
sub_.to_csv('sub.csv', index=False)

,ID,Target
0,test0_0,8
1,test0_1,8
2,test0_2,8
3,test0_3,8
4,test0_4,8
...,...,...
2337995,test18_144995,1
2337996,test18_144996,1
2337997,test18_144997,1
2337998,test18_144998,1


Target
9     235427
3     235277
10    219588
1     215766
5     214844
6     209737
7     202657
4     186938
0     183291
11    172422
8     164486
2      97567
Name: count, dtype: int64